In [ ]:
#We will be using GAN(Generative Adversarial Network) to generate images using the CIFAR-10 dataset
#GAN works by employing two functions-Generator & Discriminator
#Generator: Generates fake images starting from random noise. & Discriminator: Distinguishes between real and fake images.
#The first step would be loading the CIFAR-10 Dataset
from tensorflow.keras.datasets import cifar10

In [ ]:
#CIFAR-10 dataset has each image of 32*32 pixels, in RGB format, has 10 classes consisting of 10 image classes
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data() #Loading the training and testing data

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
print(xtrain.shape) #50000 training images and 10000 testing images
print(xtest.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
xtrain.dtype #The data type is uint8.For normalization we need to convert it to float32 and then normalize it between [-1,1]

dtype('uint8')

In [ ]:
xtrain=(xtrain.astype('float32')-127.5)/127.5 #Normalizing the data between [-1,1]


In [ ]:
#We would be first building the CNN based Generator which takes a random noise vector and generates a 32*32*3 image
#Random noise vector is like a seed which allows the generator to learn patterns and generate realistic images
#Randomness is required which makes sure that the generator learns diverse images and makes it learning holistic
#We would be using the CNN which is ideal for image based processing
from tensorflow.keras import layers,models#layers would be used to stack the neural network layers linearly
def CNN_generator():
  #Input is the random noise vector(eg.100 dimesnsion).The output of the dense layer is upsampled to 8*8*256
  model=models.Sequential()
  model.add(layers.Dense(8*8*256,input_shape=(100,)))#it signifies 4*4*256 neurons in the layer and expects an input of 100 dimensions from the previous layer
  model.add(layers.Reshape((8,8,256)))#It reshapes the input into this dimension
  model.add(layers.BatchNormalization())#Batch normaliation does some stuff like scaling on the input data.It also prevents overfitting
  model.add(layers.LeakyReLU(alpha=0.3))#This is the activation function which is applied.It simply adds non lineriaty into the network allowing it learn complex patterns

  #We can now upsample the output to 16*16*128
  model.add(layers.Conv2DTranspose(128,(5,5),strides=(2,2),padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.3))

  #We can now upsample to output to 32*32*64
  model.add(layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.3))



  #Time for the output layers 32*32*3
  model.add(layers.Conv2D(3,(5,5),activation='tanh',padding='same'))
  #We will now compile the model
  model.compile(loss='binary_crossentropy',optimizer='adam')
  return model

#We are starting with a dense layer because we it learns from the noise input vector and creates a high dimensional image and then feds into the CNN
#CNN is good at adding the details and patterns but it cannot start the sketch from scratch



In [ ]:
#We will be now buiding the discriminator model which will be downsampling starting from 32*32*3
def build_discriminator():
  model=models.Sequential()
  #We will downsample it starting from 32*32*3 to **64
  model.add(layers.Conv2D(64,(5,5),strides=(2,2),padding='same',input_shape=(32,32,3)))
  model.add(layers.LeakyReLU(alpha=0.3))
  model.add(layers.Dropout(0.4))

  #Again downsampling it
  model.add(layers.Conv2D(128,(5,5),strides=(2,2),padding='same'))
  model.add(layers.LeakyReLU(alpha=0.3))
  model.add(layers.Dropout(0.4))

  #Again downsampling it
  model.add(layers.Conv2D(256,(5,5),strides=(2,2),padding='same'))
  model.add(layers.LeakyReLU(alpha=0.3))
  model.add(layers.Dropout(0.4))

  #Now,time for the output layer.We will be using flatten to convert the multidimensional feature map to 1D vector of values
  model.add(layers.Flatten())
  model.add(layers.Dense(1,activation='sigmoid')) #It needs to input a single value as the probability
  #We will now compile the model
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model
#Upsampling happens in the generator function because it starts from a scratch and gradually adds features and patterns to the image
#Downsampling happens in the discriminator beacause it will take an image and gradually reduce the number of features in it to check for the authenticity of the image


In [ ]:
#Combine the GAN
def build_GAN(generator,discriminator):
  discriminator.trainable=False #We don't want to update the weights and biases of the discriminator model because it should not learn from the images
  model=models.Sequential([generator,discriminator])
  return model
generator=CNN_generator()
discriminator=build_discriminator()
gan=build_GAN(generator,discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import numpy as np

def train_gan(generator, discriminator, gan, data, epochs=10000, batch_size=64):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], half_batch)#The Discriminator is fed real data and is asked to classify it as real.(data.shape[0]) is the number of training images and half the number of them are fed..
        real_imgs = data[idx]
        fake_imgs = generator.predict(np.random.normal(0, 1, (half_batch, 100)))#The generator is creating images and fed into the discriminator

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)#It trains the discriminator on the
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)#It trains the generator on a single batch of data taking noise as input and geenrates images on it

        # Print progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss}]")

        # Save generated images
        if epoch % 1000 == 0:
            save_generated_images(epoch, generator)

# Save images
import matplotlib.pyplot as plt

def save_generated_images(epoch, generator, examples=25, dim=(5, 5)):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    plt.figure(figsize=(10, 10))
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_images_epoch_{epoch}.png")
    plt.close()

train_gan(generator,discriminator,gan,xtrain)



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step


0 [D loss: 0.7042796611785889 | D acc.: 49.22%] [G loss: [array(0.70051515, dtype=float32), array(0.70051515, dtype=float32), array(0.70051515, dtype=float32), array(0.546875, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 